FAKE NEWS DETECTION

WORK FLOW:

1) Extracting news data
2) Data pre processing
3) Train and test splitting
4) Logistic regression will be used as this is a binary classification project i.e the news will be either real or fake
5) We get a trained model and we can do some evaluation on this model and find accuracy of the model using test data
6) We feed new data in this model and predict whether news is real or fake
   

Dataset Description1) id: unique id for a news article
2) title: the title of the news article
3) author: author of the news article
4) text: description of the article; could be incomplete
5) label: a label that marks whether the news is real or fake

1:Fake News 
0:Real News

IMPORTING NECESSARY LIBRARIES


In [45]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 


In [75]:
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [49]:
print(stopwords.words('english')) #printing stopwords in english

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PRE-PROCESSING

In [51]:

data=pd.read_csv(r"C:\Users\Varni\Downloads\train.csv.zip")
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [53]:
data.shape

(20800, 5)

In [55]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [56]:
data=data.fillna('')

In [57]:
data['information']=data['author']+'    '+ data['title']
data['information']

0        Darrell Lucus    House Dem Aide: We Didn’t Eve...
1        Daniel J. Flynn    FLYNN: Hillary Clinton, Big...
2        Consortiumnews.com    Why the Truth Might Get ...
3        Jessica Purkiss    15 Civilians Killed In Sing...
4        Howard Portnoy    Iranian woman jailed for fic...
                               ...                        
20795    Jerome Hudson    Rapper T.I.: Trump a ’Poster ...
20796    Benjamin Hoffman    N.F.L. Playoffs: Schedule,...
20797    Michael J. de la Merced and Rachel Abrams    M...
20798    Alex Ansary    NATO, Russia To Hold Parallel E...
20799           David Swanson    What Keeps the F-35 Alive
Name: information, Length: 20800, dtype: object

In [58]:
#we will compare information column with label column
x=data.drop(columns='label',axis=1)
y=data['label']
x


,id,title,author,text,information
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Eve...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Sing...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fic...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster ...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule,..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams M...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel E..."


In [59]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

STEMMING


In [60]:
port_stem=PorterStemmer()

In [61]:
def stemming(information):
    stemmed_information=re.sub('[^a-zA-Z]',' ',information)
    stemmed_information= stemmed_information.lower()
    stemmed_information= stemmed_information.split()
    stemmed_information= [port_stem.stem(word) for word in stemmed_information if not word in stopwords.words('english')]
    stemmed_information=' '.join(stemmed_information)
    return stemmed_information

data['information']=data['information'].apply(stemming)
print(data['information'])



0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: information, Length: 20800, dtype: object


In [62]:

x=data['information'].values
y=data['label'].values

In [63]:
x

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [64]:
y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [65]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)


In [66]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

 Splitting data into training and testing

In [67]:
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.2,stratify=y, random_state=2)


Training the model

In [68]:
model=LogisticRegression()

In [69]:
model.fit(x_train, y_train)


LogisticRegression()

Evaluation

accuracy score

In [70]:
#accuracy score on the training data
x_train_prediction= model.predict(x_train)
training_data_accuracy=accuracy_score(x_train_prediction, y_train)


In [71]:
print('accuracy score of training data: ', training_data_accuracy)


accuracy score of training data:  0.9865985576923076


In [72]:
#accuracy score on the testing data
x_test_prediction= model.predict(x_test)
testing_data_accuracy=accuracy_score(x_test_prediction, y_test)


In [73]:
print('accuracy score of testing data: ', testing_data_accuracy)


accuracy score of testing data:  0.9790865384615385


Making a predictive system


In [74]:
x_new = x_test[0] #selecting 1st row from x_test

prediction=model.predict(x_new)
print(prediction)

if (prediction[0]==0):
    print('The news is real')
else:
    print('The news is fake')
    


[1]
The news is fake
